In [1]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? υ
Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [28]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
from os.path import expanduser, join
from re import findall
from pandas import DataFrame, to_datetime, merge
from time import time

from cassandra import ConsistencyLevel
from cassandra.query import SimpleStatement

In [3]:
home = expanduser('~')
mypath = f'{home}/Downloads/BigData/'
deromalized_path = join(mypath, 'Datastax_denormalized/').replace('\\', '/')

## Connect to DataStax

In [4]:
cloud_config= {'secure_connect_bundle': mypath+'secure-connect-bigdataproject2022.zip'}

with open(mypath+'GeneratedToken.csv', 'r', encoding='utf-8') as file:
    try:
        content = file.readlines()        
    except FileNotFoundError:
        raise('File not found')

tokens = findall(r'"(.*?)"', content[1])
client_id = tokens[0]
client_secret = tokens[1]

In [5]:
keyspace = 'big_data'
auth_provider = PlainTextAuthProvider(client_id, client_secret)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect(keyspace)

## Q1

## Retrieve data for Q1

In [144]:
def select_from_popular_movies_by_date(start_date, end_date):
    sd = to_datetime(start_date)
    ed = to_datetime(end_date)
    if sd.year == ed.year:
        months = ','.join([str(i) for i in range(sd.month, ed.month+1)])
        Q1 = f"SELECT movieid, title, genres, rating FROM popular_movies_by_date WHERE year={sd.year} AND month IN({months}) AND day>={sd.day} AND day<={ed.day};"
        
        query = SimpleStatement(Q1, consistency_level=ConsistencyLevel.ONE)
        query_results = session.execute(Q1)
        
        movies = DataFrame([(row.movieid, row.title, row.genres, row.rating) for row in query_results], columns=['movieid', 'title', 'genres', 'rating'])
        u_movies = DataFrame(movies.groupby(by='movieid')['rating'].mean())
        u_movies.reset_index(inplace=True)
        u_movies.sort_values(by='rating', ascending=False, inplace=True)
        movies = merge(left=u_movies, right=movies.loc[:, 'movieid':'genres'], left_on='movieid', right_on='movieid')
        
        for index, row in movies.iterrows():
            print(row.title, row.rating, row.genres)
            if index == 4:
                break
    else:
        print("Invalid date range!")

In [145]:
total = 0
for _ in range(10):
    s = time()
    select_from_popular_movies_by_date('2015-01-01', '2015-01-15')
    e = time()
    total += (e-s)
    print((e-s)*1000, '\n')
print(total/10)

Wonder Bar 5.0 Comedy|Crime|Drama|Musical|Romance
The Best Years of Our Lives 5.0 Drama|War
Still Bill 5.0 Documentary
The Brown Bunny 5.0 Drama
Secretariat 5.0 Adventure|Drama
3232.62882232666 

Wonder Bar 5.0 Comedy|Crime|Drama|Musical|Romance
The Best Years of Our Lives 5.0 Drama|War
Still Bill 5.0 Documentary
The Brown Bunny 5.0 Drama
Secretariat 5.0 Adventure|Drama
3487.502336502075 

Wonder Bar 5.0 Comedy|Crime|Drama|Musical|Romance
The Best Years of Our Lives 5.0 Drama|War
Still Bill 5.0 Documentary
The Brown Bunny 5.0 Drama
Secretariat 5.0 Adventure|Drama
3200.0222206115723 

Wonder Bar 5.0 Comedy|Crime|Drama|Musical|Romance
The Best Years of Our Lives 5.0 Drama|War
Still Bill 5.0 Documentary
The Brown Bunny 5.0 Drama
Secretariat 5.0 Adventure|Drama
3254.558801651001 

Wonder Bar 5.0 Comedy|Crime|Drama|Musical|Romance
The Best Years of Our Lives 5.0 Drama|War
Still Bill 5.0 Documentary
The Brown Bunny 5.0 Drama
Secretariat 5.0 Adventure|Drama
3253.3631324768066 

Wonder Bar 5.0

## Q2

## Retrieve data for Q2

In [152]:
def select_from_movies_by_keyword(keyword='toy', limit=5):
    Q2 = f"SELECT title, rating, genres FROM movies_by_keyword WHERE keyword='{keyword.lower()}' LIMIT {limit};"
    query = SimpleStatement(Q2, consistency_level=ConsistencyLevel.ONE)
    query_results = session.execute(Q2)
    rows = session.execute(Q2)
    for row in rows:
        print(f"Title: {row.title} Rating: {row.rating:.1f} Genres: {row.genres}")

In [153]:
total = 0

for keyword in ['star', 'toy', 'dead', 'lost', 'highway', 'blue', 'velvet', 'dance', 'dark']:
    s = time()
    select_from_movies_by_keyword(keyword=keyword)
    e = time()
    total += (e-s)
    print((e-s)*1000, '\n')
print(total/10)

Title: Star Wars: Episode IV - A New Hope Rating: 4.2 Genres: Action|Adventure|Sci-Fi
Title: Star Wars: Episode V - The Empire Strikes Back Rating: 4.2 Genres: Action|Adventure|Sci-Fi
Title: Lucky Star Rating: 4.2 Genres: Drama|Romance
Title: Lone Star Rating: 4.1 Genres: Drama|Mystery|Western
Title: The Cloud-Capped Star Rating: 4.1 Genres: Drama
256.90650939941406 

Title: Toy Story 3 Rating: 4.0 Genres: Adventure|Animation|Children|Comedy|Fantasy|IMAX
Title: Toy Story Rating: 3.9 Genres: Adventure|Animation|Children|Comedy|Fantasy
Title: Toy Story 2 Rating: 3.8 Genres: Adventure|Animation|Children|Comedy|Fantasy
Title: Toy Story of Terror Rating: 3.6 Genres: Animation|Children|Comedy
Title: The Toy Rating: 3.5 Genres: Comedy
187.46185302734375 

Title: Dead Time: Kala Rating: 4.5 Genres: Crime|Fantasy|Film-Noir|Thriller
Title: The Dead Man and Being Happy Rating: 4.2 Genres: Drama
Title: Nobody Will Speak of Us When Were Dead Rating: 4.0 Genres: Drama
Title: Dead on Time Rating: 4.0

# Q3

## Retrieve data for Q3

In [158]:
def select_from_movies_by_genre(genre, limit=10):
    Q3 = f"SELECT title, avg_rating, release_date FROM movies_by_genre WHERE genre='{genre}' ORDER BY release_date ASC LIMIT {limit};"
    SimpleStatement(Q3, consistency_level=ConsistencyLevel.ONE)
    query_results = session.execute(Q3)
    rows = session.execute(Q3)
    for row in rows:
        print(f"Title: {row.title} Rating: {row.avg_rating:.1f} Release Date: {row.release_date}")    

In [159]:
total = 0
for genre in ['Adventure', 'Comedy', 'Horror', 'Mystery', 'Musical', 'Action', 'Drama', 'Romance', 'Thriller', 'Fantasy']:
    s = time()
    select_from_movies_by_genre(genre, limit=5)
    e = time()
    total += (e-s)
    print((e-s)*1000, '\n')
print(total/10)

Title: A Trip to the Moon Rating: 3.7 Release Date: 1902
Title: The Last Days of Pompeii Rating: 2.5 Release Date: 1913
Title: Judith of Bethulia Rating: 3.3 Release Date: 1914
Title: Cabiria Rating: 3.1 Release Date: 1914
Title: Vampires, Les Rating: 3.4 Release Date: 1915
237.1048927307129 

Title: Tables Turned on the Gardener Rating: 2.2 Release Date: 1895
Title: The Black Devil Rating: 2.5 Release Date: 1905
Title: Those Awful Hats Rating: 2.8 Release Date: 1909
Title: The Cameramans Revenge Rating: 3.9 Release Date: 1912
Title: Gertie the Dinosaur Rating: 3.5 Release Date: 1914
187.37077713012695 

Title: Frankenstein Rating: 3.5 Release Date: 1910
Title: The Student of Prague Rating: 3.2 Release Date: 1913
Title: The Avenging Conscience Rating: 3.3 Release Date: 1914
Title: Vampires, Les Rating: 3.4 Release Date: 1915
Title: I Accuse Rating: 3.9 Release Date: 1919
181.58793449401855 

Title: Blind Justice Rating: 3.5 Release Date: 1916
Title: Eerie Tales Rating: 2.5 Release Date

## Q4

## Retrieve data for Q4

In [166]:
def select_from_movie_info_by_title(session, title):
    Q4 = f"SELECT title, genres, tag, avg_rating AS avg FROM movie_info_by_title WHERE title='{title}' ORDER BY tag_count LIMIT 5;"
    SimpleStatement(Q4, consistency_level=ConsistencyLevel.ONE)
    query_results = session.execute(Q4)
    rows = session.execute(Q4)
    print(f"Title: {rows[0].title} \nRating: {rows[0].avg:.1f} \nGenres: {rows[0].genres}")
    print(f"Most popular tags: ")
    for row in rows:
        print(row.tag)

In [167]:
total = 0
for title in ['Jumanji', 'Agora', 'The Great Santini', 'Paranormal Activity 3', 'Clockstoppers', 'Twilight of the Ice Nymphs', 'Stuart: A Life Backward', 'Diary of a Wimpy Kid', 'Washington Square', 'Godzilla vs. Megaguirus']:
    s = time()
    select_from_movie_info_by_title(session, title)
    e = time()
    total += (e-s)
    print((e-s)*1000, '\n')
    
print(total/10)

Title: Jumanji 
Rating: 3.2 
Genres: Adventure|Children|Fantasy
Most popular tags: 
time travel
time
see alsoZathura
see also:Zathura
scary
181.27965927124023 

Title: Agora 
Rating: 3.5 
Genres: Adventure|Drama|Romance
Most popular tags: 
unrequited love
slavery
religion
rationality
philosophy
187.46113777160645 

Title: The Great Santini 
Rating: 3.8 
Genres: Drama
Most popular tags: 
perrot library
fatherhood
Robert Duval
Lewis John Carlino
Betamax
181.35857582092285 

Title: Paranormal Activity 3 
Rating: 2.9 
Genres: Horror
Most popular tags: 
vhs
horror
haunted house
ghosts
franchise
187.45160102844238 

Title: Clockstoppers 
Rating: 2.7 
Genres: Action|Adventure|Sci-Fi|Thriller
Most popular tags: 
classic
Paula GarcÃ©s
Paula GarcÃs
Jonathan Frakes
187.1945858001709 

Title: Twilight of the Ice Nymphs 
Rating: 2.3 
Genres: Fantasy
Most popular tags: 
stylized
silly title
ostrich farm
mad scientist
journey home
225.16489028930664 

Title: Stuart: A Life Backward 
Rating: 4.5 
Genr

## Q5

## Retrieve data for Q5

In [172]:
def select_from_movies_by_tag(tag, limit=5):
    Q5 = f"SELECT title, avg_rating as avg, genres FROM movies_by_tag WHERE tag='{tag}' ORDER BY avg_rating DESC LIMIT {limit};"
    SimpleStatement(Q5, consistency_level=ConsistencyLevel.ALL)
    query_results = session.execute(Q5)
    rows = session.execute(Q5)
    for row in rows:
        print(f"Title: {row.title} Rating: {row.avg:.1f}")

In [173]:
total = 0
for tag in ['daredevil', 'loose ends', 'musical sucks.', 'jeff bleckner',
            'von bulow', 'gargamel', 'dance', 'relevant today', 'chernobyl']:
    s = time()
    select_from_movies_by_tag('comedy')
    e = time()
    total += (e-s)
    print((e-s)*1000, '\n')
print(total/10)

Title: The Foster Brothers Rating: 5.0
Title: Johnny Express Rating: 4.4
Title: The Lizard Rating: 4.3
Title: Pulp Fiction Rating: 4.2
Title: Wallace & Gromit: A Close Shave Rating: 4.2
182.71636962890625 

Title: The Foster Brothers Rating: 5.0
Title: Johnny Express Rating: 4.4
Title: The Lizard Rating: 4.3
Title: Pulp Fiction Rating: 4.2
Title: Wallace & Gromit: A Close Shave Rating: 4.2
187.35313415527344 

Title: The Foster Brothers Rating: 5.0
Title: Johnny Express Rating: 4.4
Title: The Lizard Rating: 4.3
Title: Pulp Fiction Rating: 4.2
Title: Wallace & Gromit: A Close Shave Rating: 4.2
257.89523124694824 

Title: The Foster Brothers Rating: 5.0
Title: Johnny Express Rating: 4.4
Title: The Lizard Rating: 4.3
Title: Pulp Fiction Rating: 4.2
Title: Wallace & Gromit: A Close Shave Rating: 4.2
255.4035186767578 

Title: The Foster Brothers Rating: 5.0
Title: Johnny Express Rating: 4.4
Title: The Lizard Rating: 4.3
Title: Pulp Fiction Rating: 4.2
Title: Wallace & Gromit: A Close Shave